In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MetricHead(nn.Module):
    def __init__(self, in_channels):
        super(MetricHead, self).__init__()
        
        # Riduzione a 2 canali con una convoluzione
        self.conv = nn.Conv2d(in_channels, 2, kernel_size=1)
        
        # Vettori normalizzati da apprendere (13 vettori di 2 dimensioni)
        self.learnable_vectors = nn.Parameter(torch.randn(13, 2))

    def forward(self, x):
        # Riduci i canali a 2 e normalizza
        x = self.conv(x)  # (b, 2, h, w)
        x = F.normalize(x, p=2, dim=1)

        # Normalizza i vettori appresi direttamente nel forward
        normalized_vectors = F.normalize(self.learnable_vectors, p=2, dim=1)

        # Calcolo delle distanze rispetto ai vettori appresi
        b, c, h, w = x.shape

        # Risagomare per calcolare la distanza
        x_flat = x.view(b, c, -1).permute(0, 2, 1)  # (b, h*w, 2)

        # Calcola la distanza per ciascun vettore
        distances = []
        for v in normalized_vectors:
            v = v.unsqueeze(0).expand(b, -1)
            dist = torch.norm(x_flat - v, dim=2)
            distances.append(dist.view(b, h, w))

        # Stack delle distanze (b, 13, h, w)
        distances = torch.stack(distances, dim=1)
        return distances

# Esempio di utilizzo
model = MetricHead(in_channels=3)
x = torch.randn(1, 3, 32, 32)
distances = model(x)
print(distances.shape)

torch.Size([1, 1024, 2])
torch.Size([1, 1024, 2])
torch.Size([1, 1024, 2])
torch.Size([1, 1024, 2])
torch.Size([1, 1024, 2])
torch.Size([1, 1024, 2])
torch.Size([1, 1024, 2])
torch.Size([1, 1024, 2])
torch.Size([1, 1024, 2])
torch.Size([1, 1024, 2])
torch.Size([1, 1024, 2])
torch.Size([1, 1024, 2])
torch.Size([1, 1024, 2])
torch.Size([1, 13, 32, 32])


In [1]:
import os

In [2]:
def find_metrics(log_file):
    metrics = {}

    for line in log_file.readlines():
        if len(line):
            tokens = line.split()
            
            # IoU metrics
            if 'mIoU_no_zero:' in tokens:
                metrics['mIoU'] = float(tokens[4])*100
            elif 'classe' in tokens and '3:' in tokens:
                metrics['minIoU'] = float(tokens[2])*100
            elif 'maxIoU:' in tokens:
                metrics['maxIoU'] = float(tokens[4])*100
            # anomaly metrics
            if 'AUPR:' in tokens:
                metrics[tokens[3].lower()] = float(tokens[5])*100
    return metrics

def find_parameters(log_file):
    params = {}

    flag = True
    for line in log_file.readlines():
        if len(line):
            tokens = line.split()
            
            if flag:
                if 'Dataset' in tokens:
                    flag = False
                else:
                    params[tokens[3]] = tokens[5]
            else:
                break
    return params

def prepare_metrics(metrics):
    stringa = ''
    stringa += f"{round(metrics['mIoU'],2)};"
    stringa += f"{round(metrics['minIoU'],2)};"
    stringa += f"{round(metrics['maxIoU'],2)};"
    stringa += f"{round(metrics['msp'],2)};"
    stringa += f"{round(metrics['maxlog'],2)};"
    stringa += f"{round(metrics['entropy'],2)};"
    stringa += f"{round(metrics['energy'],2)}"
    return stringa.replace('.',',')

def prepare_parameters(parameters):
    mappa = {'True': '✓', 'False': '𐄂'}

    stringa = ''
    stringa += f"{mappa[parameters['biases']]};"
    stringa += f"{mappa[str(bool(int(parameters['focal_loss'])))]};"
    stringa += f"{mappa[parameters['class_weights']]};"
    stringa += f"{mappa[parameters['norm_weights']]};"
    try:
        stringa += f"{mappa[str(bool(int(parameters['mlp'])))]};"
    except:
        stringa += f"{mappa['False']};"
    stringa += f"{parameters['activation']};"
    return stringa

In [3]:
folder = '/raid/homespace/piecestola/space/ML4CV/results_7'

texts = list()
order = list()
for file in os.listdir(folder):
    try:
        path_train = os.path.join(folder,file,'train.log')
        path_test = os.path.join(folder,file,'test.log')
        
        params = find_parameters(open(path_train,'r'))
        metrics = find_metrics(open(path_test,'r'))

        params = prepare_parameters(params)
        metrics = prepare_metrics(metrics)

        order.append(int(file.split('_')[1]))

        text = params+metrics
        text = text.replace(';',' & ')
        text = text.replace('softmax','\\text{softmax}')
        text = text.replace('sigmoid','\\text{sigmoid}')
        text += ' \\\\'
        texts.append(text)
    except:
        print("Skipped",file)

texts = [v for _, v in sorted(zip(order, texts))]

print('Letti correttamente:',end=' ')
for k, v in sorted(zip(order, texts)):
    print(k,end=' ')
print('\n')

print('\\begin{array}{|cccccc|ccc|cccccc|}')
print('\\hline')
print('\\text{Biases} & \\text{Focal Loss} & \\text{Class weights} & \\text{Norm weights} & \\text{MLP} & \\text{Activation} & \\text{mIoU} & \\text{minIoU} & \\text{maxIoU} & \\text{msp} & \\text{maxlogit} & \\text{entropy} & \\text{energy} \\\\')
print('\\hline')

maxs = [0,0,0,0,0,0,0]
maxs_id = [0,0,0,0,0,0,0]
for i, text in enumerate(texts):
    tokens = text.split()
    for j in range(12,26,2):
        temp = float(tokens[j].replace(',','.'))
        if temp > maxs[j//2-6]:
            maxs[j//2-6] = temp
            maxs_id[j//2-6] = i

for i, text in enumerate(texts):
    tokens = text.split()
    for k, id in enumerate(maxs_id):
        if id == i:
            tokens[(k+6)*2] = '\\underline{'+tokens[(k+6)*2]+'}'
    print(' '.join(tokens))

print('\\hline')
print('\\end{array}')

Letti correttamente: 1 4 5 6 7 8 9 10 

\begin{array}{|cccccc|ccc|cccccc|}
\hline
\text{Biases} & \text{Focal Loss} & \text{Class weights} & \text{Norm weights} & \text{MLP} & \text{Activation} & \text{mIoU} & \text{minIoU} & \text{maxIoU} & \text{msp} & \text{maxlogit} & \text{entropy} & \text{energy} \\
\hline
𐄂 & 𐄂 & 𐄂 & 𐄂 & 𐄂 & \text{softmax} & \underline{65,41} & \underline{10,97} & \underline{97,16} & 10,79 & \underline{15,38} & \underline{13,71} & \underline{15,69} \\
𐄂 & 𐄂 & 𐄂 & 𐄂 & 𐄂 & \text{softmax} & 59,93 & 8,44 & 96,15 & 9,68 & 11,69 & 11,5 & 11,55 \\
𐄂 & 𐄂 & 𐄂 & 𐄂 & 𐄂 & \text{sigmoid} & 57,49 & 8,42 & 96,0 & 7,41 & 7,59 & 8,5 & 7,21 \\
𐄂 & 𐄂 & 𐄂 & ✓ & 𐄂 & \text{softmax} & 50,61 & 4,23 & 95,1 & 10,14 & 3,1 & 8,24 & 0,54 \\
𐄂 & 𐄂 & 𐄂 & ✓ & 𐄂 & \text{softmax} & 48,49 & 3,69 & 96,1 & 8,69 & 3,54 & 3,3 & 0,56 \\
𐄂 & 𐄂 & ✓ & ✓ & 𐄂 & \text{softmax} & 36,4 & 3,34 & 91,81 & 3,53 & 2,94 & 5,32 & 0,78 \\
𐄂 & 𐄂 & 𐄂 & ✓ & 𐄂 & \text{softmax} & 60,86 & 9,24 & 96,27 & \underline{11,03} &